<a href="https://colab.research.google.com/github/Qu1nnD/CS290/blob/main/Q%26Cproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree
import math
from sklearn.model_selection import train_test_split
from statistics import mean, stdev
from sklearn.metrics import accuracy_score

In [87]:
shrooms = pd.read_csv("https://raw.githubusercontent.com/Qu1nnD/CS290/main/data/mushroom_cleaned.csv") # categorical dataset
StudentPerform = pd.read_csv("https://github.com/Charlee0616/Data-Mining/raw/main/StudentPerformanceFactors.csv") # numerical dataset

In [88]:
def Attribute_selection_method(dataset, target, method, classification):
  if(method == "classification"): #method chosen was classification
    features = dataset.columns[dataset.columns != target]
    entropys=[]
    for attribute in features:
      entropy = 0
      overall = len( dataset )
      vals = dataset[attribute].unique()
      if(classification =="entropy"): #classifcation was chosen and entropy was chosen
        for val in vals:
          if(len(dataset[attribute].unique())<20):
            subset_size = len(dataset[ dataset[attribute] == val ])
            weight = subset_size / overall
            props = dataset[ dataset[attribute] == val ][target].value_counts( normalize=True )
            for p in props.array:
              entropy =  entropy - weight*(p*math.log2(p))
        else: #classification was chose and Gini was chosen
          entropy = 0
          left = dataset[ dataset[attribute] <= val ][ [attribute,target] ]
          props = left[ target ].value_counts( normalize = True )
          weight = len( left ) / overall
          for prop in props.array:
            entropy = entropy - weight*prop*math.log2( prop )
          right = dataset[ dataset[attribute] > val ][ [attribute,target] ]
          props = right[ target ].value_counts( normalize = True )
          weight = len( right ) / overall
          for prop in props.array:
            entropy = entropy - weight*prop*math.log2( prop )
          entropys.append(entropy)
      else:
        #for val in vals:
          #props = dataset[ dataset[attribute] == val ][target].value_counts( normalize=True )
          #for prop in props.array:
        entropy = 1 - sum([prop ** 2])
          #entropy += weight * entropy
        entropys.append(entropy)
    minEnt=min(entropys)
    ind=entropys.index(minEnt)
    return features[ind]
  else: # method is Naive Bayes Classifier/ regressor
    #separate by class
    separated = {}
    for i in range(len(dataset)):
      vector = dataset[i]
      class_value = vector[-1]
      if (class_value not in separated):
        separated[class_value] = []
      separated[class_value].append(vector)

    #Summarize statistivc
    summaries = {}
    for class_value, instances in separated.items():
      summaries[class_value] = [(mean(col), stdev(col), len(col)) for col in zip(*instances)]

    predictions = []
    #put test into the method
    for row in test:
        total_rows = sum([summaries[label][0][2] for label in summaries])
        probabilities = {}
        for class_value, class_summaries in summaries.items():
          probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
          for i in range(len(class_summaries)):
            mean_val, stdev_val, _ = class_summaries[i]
            exponent = math.exp(-((row[i]-mean_val)**2 / (2 * stdev_val**2 )))
            probabilities[class_value] *=(1 / (math.sqrt(2 * math.pi) * stdev_val)) * exponent
        #determine the best label
        best_label, best_prob = None, -1
        for class_value, probability in probabilities.items():
          if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value

        predictions.append(best_label)
    return(predictions)

In [89]:
#Testing and Training data for Shrooms dataset
C_train_set, C_test_set = train_test_split(shrooms, test_size=0.2)
C_train_set = C_train_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color', 'stem-height', 'stem-width', 'stem-color', 'class']]
C_test_set = C_test_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color', 'class']]
C_X_train = C_train_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color']]
C_y_train = C_train_set["class"]
C_X_test = C_test_set[['cap-diameter', 'cap-shape', 'gill-attachment', 'gill-color','stem-height', 'stem-width', 'stem-color']]
C_y_test = C_test_set["class"]
cat_attributes=[]
num_attributes=[]
for col in C_X_train.columns:
  if len(C_X_train[col].unique()) < 20:
    cat_attributes.append(col)
  else:
    num_attributes.append(col)
trf = [ ('num', StandardScaler(), num_attributes),
       ('cat', OneHotEncoder( handle_unknown='ignore'), cat_attributes) ]
col_transform = ColumnTransformer( transformers = trf )
pipeline = Pipeline( steps = [('pre', col_transform),
 ('clf', DecisionTreeClassifier())])

In [90]:
R_train_set, R_test_set = train_test_split(StudentPerform, test_size=0.2)
num_attributes = R_X_train.select_dtypes( include = ['int64','float64']).columns
R_X_train = R_train_set[num_attributes[num_attributes!="Exam_Score"]]
R_y_train = R_train_set["Exam_Score"]
R_X_test = R_test_set[num_attributes[num_attributes!="Exam_Score"]]
R_y_test = R_test_set["Exam_Score"]

In [91]:
# Categorical Dataset Test
# Gini test
Cat_G=Attribute_selection_method(shrooms,"class","classification","gini")
Cat_G

UnboundLocalError: local variable 'prop' referenced before assignment

In [ ]:
# Classification Tree Test for comparison
DecisionTreeClassifier()

In [ ]:
# Categorical Dataset Test
# Entropy test
Cat_E=Attribute_selection_method(shrooms,"class","classification","entropy")
Cat_E

In [ ]:
# Classification Tree Test for comparison
DecisionTreeClassifier('entropy')

In [ ]:
# Numerical Dataset Test
numT=Attribute_selection_method(StudentPerform,"Exam_Score","regression","nothing")
numT

In [92]:
# Regression Tree Test for comparison
Regression_tree_model = DecisionTreeRegressor()
Regression_tree_model.fit(R_X_train, R_y_train)
R_y_pred_dt = Regression_tree_model.predict(R_X_test)
print("Decision Tree Regressor Model Performance:")
print(f"Accuracy: {accuracy_score(R_y_test, R_y_pred_dt):.2f}")

Decision Tree Regressor Model Performance:


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets